In [1]:
import pandas as pd

df = pd.read_csv("Bi-Annual_Pedestrian_Counts_20260204.csv")

df.head()


,the_geom,OBJECTID,Loc,Borough,Street_Nam,From_Stree,To_Street,Iex,May07_AM,May07_PM,...,Oct23_MD,June24_AM,June24_PM,June24_MD,Oct24_AM,Oct24_PM,Oct24_MD,May25_AM,May25_PM,May25_MD
0,POINT (-73.90459140730678 40.87919896648574),1,1,Bronx,Broadway,West 231st Street,Naples Terrace,N,1.189,4.094,...,1.745,901.000,3.482,2.052,1.031,4.097,2229,811.000,2.420,2.224
1,POINT (-73.92188432870218 40.82662794123292),2,2,Bronx,East 161st Street,Gra Concourse,Sheridan Avenue,Y,1.511,3.184,...,1.287,2.113,4.155,1.493,3.016,4.685,1655,2.928,4.168,2.468
2,POINT (-73.89535781584335 40.86215460031517),3,3,Bronx,East Fordham Road,Valentine Avenue,Tiebout Avenue,Y,1.832,12.311,...,2.453,1.485,7.125,3.819,1.689,6.405,4417,1.599,4.936,4.688
3,POINT (-73.87892467324478 40.8812869959873),4,4,Bronx,East Gun Hill Road,Bainbridge Avenue,Rochambeau Avenue,N,764.000,2.673,...,807.000,1.394,2.023,1.190,1.495,2.474,1073,1.261,1.779,1.133
4,POINT (-73.88956389732787 40.844636776717664),5,5,Bronx,East Tremont Avenue,Prospect Avenue,Clinton Avenue,N,650.000,2.538,...,1.115,550.000,1.455,1.563,725.000,2.329,1888,714.000,1.585,1.522


In [2]:
df.columns

Index(['the_geom', 'OBJECTID', 'Loc', 'Borough', 'Street_Nam', 'From_Stree',
       'To_Street', 'Iex', 'May07_AM', 'May07_PM',
       ...
       'Oct23_MD', 'June24_AM', 'June24_PM', 'June24_MD', 'Oct24_AM',
       'Oct24_PM', 'Oct24_MD', 'May25_AM', 'May25_PM', 'May25_MD'],
      dtype='object', length=113)

In [3]:
# Kopya oluşturalım (orijinal bozulmasın)
df2 = df.copy()

# Zamanla ilgili sütunları seçelim
time_cols = [c for c in df2.columns if '_' in c and c[:3] in ['May','Jun','Oct']]

time_cols[:10]


['May07_AM',
 'May07_PM',
 'May07_MD',
 'May08_AM',
 'May08_PM',
 'May08_MD',
 'May09_AM',
 'May09_PM',
 'May09_MD',
 'May10_AM']

In [4]:
# ID olarak kullanacağımız sütunlar
id_cols = [
    'Borough',
    'Street_Nam',
    'From_Stree',
    'To_Street',
    'the_geom'
]

# Melt işlemi (wide → long)
long_df = df2.melt(
    id_vars=id_cols,
    value_vars=time_cols,
    var_name='date_time',
    value_name='pedestrian_count'
)

long_df.head()


,Borough,Street_Nam,From_Stree,To_Street,the_geom,date_time,pedestrian_count
0,Bronx,Broadway,West 231st Street,Naples Terrace,POINT (-73.90459140730678 40.87919896648574),May07_AM,1.189
1,Bronx,East 161st Street,Gra Concourse,Sheridan Avenue,POINT (-73.92188432870218 40.82662794123292),May07_AM,1.511
2,Bronx,East Fordham Road,Valentine Avenue,Tiebout Avenue,POINT (-73.89535781584335 40.86215460031517),May07_AM,1.832
3,Bronx,East Gun Hill Road,Bainbridge Avenue,Rochambeau Avenue,POINT (-73.87892467324478 40.8812869959873),May07_AM,764.000
4,Bronx,East Tremont Avenue,Prospect Avenue,Clinton Avenue,POINT (-73.88956389732787 40.844636776717664),May07_AM,650.000


In [5]:
# Month, Year, Period ayıralım

long_df['month'] = long_df['date_time'].str[:3]
long_df['year'] = '20' + long_df['date_time'].str[3:5]
long_df['period'] = long_df['date_time'].str[-2:]

long_df[['date_time','month','year','period']].head()


,date_time,month,year,period
0,May07_AM,May,2007,AM
1,May07_AM,May,2007,AM
2,May07_AM,May,2007,AM
3,May07_AM,May,2007,AM
4,May07_AM,May,2007,AM


In [6]:
long_df['pedestrian_count'].describe()


count     7866.000000
mean       174.117272
std        545.647332
min          0.000000
25%          2.161000
50%          4.792000
75%        144.750000
max      15831.000000
Name: pedestrian_count, dtype: float64

In [7]:
# 50'nin altındakileri atalım (noise gibi düşün)
clean_df = long_df[long_df['pedestrian_count'] >= 50]

clean_df['pedestrian_count'].describe()


count     2261.000000
mean       594.476704
std        887.677418
min         50.000000
25%        234.000000
50%        455.000000
75%        729.000000
max      15831.000000
Name: pedestrian_count, dtype: float64

In [8]:
# 3 seviye oluşturalım
clean_df['crowd_level'] = pd.qcut(
    clean_df['pedestrian_count'],
    q=3,
    labels=['Low', 'Medium', 'High']
)

clean_df[['pedestrian_count','crowd_level']].head(10)


/var/folders/w9/0mtgl20j2dxdztqhj2pb2bx80000gn/T/ipykernel_2614/2257630166.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['crowd_level'] = pd.qcut(


,pedestrian_count,crowd_level
3,764.0,High
4,650.0,High
5,489.0,Medium
6,703.0,High
8,856.0,High
9,504.0,Medium
11,983.0,High
12,357.0,Medium
18,739.0,High
19,329.0,Medium


In [10]:
clean_df = clean_df.copy()

clean_df.loc[:, 'crowd_level'] = pd.qcut(
    clean_df['pedestrian_count'],
    q=3,
    labels=['Low','Medium','High']
)

In [11]:
# Gerekli kolonları seçelim
ml_df = clean_df[[
    'Borough',
    'month',
    'year',
    'period',
    'pedestrian_count',
    'crowd_level'
]].copy()

ml_df.head()


,Borough,month,year,period,pedestrian_count,crowd_level
3,Bronx,May,2007,AM,764.0,High
4,Bronx,May,2007,AM,650.0,High
5,Bronx,May,2007,AM,489.0,Medium
6,Bronx,May,2007,AM,703.0,High
8,Brooklyn,May,2007,AM,856.0,High


In [12]:
# X ve y ayıralım
X = ml_df.drop('crowd_level', axis=1)
y = ml_df['crowd_level']

# One-hot encoding
X_encoded = pd.get_dummies(X)

X_encoded.head()


,pedestrian_count,Borough_Bronx,Borough_Brooklyn,Borough_East River Bridges,Borough_Harlem River Bridges,Borough_Manhattan,Borough_Queens,Borough_Staten Isla,month_Jun,month_May,...,year_2021,year_2022,year_2023,year_2024,year_2025,year_20e2,period_AM,period_MD,period_PM,period_pM
3,764.0,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
4,650.0,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
5,489.0,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
6,703.0,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
8,856.0,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)


Train size: (1808, 35)
Test size: (453, 35)


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Model
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=12,
    random_state=42
)

# Train
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

        High       1.00      1.00      1.00       151
         Low       1.00      1.00      1.00       151
      Medium       1.00      1.00      1.00       151

    accuracy                           1.00       453
   macro avg       1.00      1.00      1.00       453
weighted avg       1.00      1.00      1.00       453



In [15]:
# pedestrian_count'u çıkarıyoruz (leakage önleme)
X = ml_df.drop(['crowd_level','pedestrian_count'], axis=1)
y = ml_df['crowd_level']

# Encode
X_encoded = pd.get_dummies(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Train again
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=12,
    random_state=42
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.49448123620309054
              precision    recall  f1-score   support

        High       0.56      0.55      0.56       151
         Low       0.54      0.58      0.56       151
      Medium       0.38      0.35      0.36       151

    accuracy                           0.49       453
   macro avg       0.49      0.49      0.49       453
weighted avg       0.49      0.49      0.49       453



In [16]:
# Period'u sayıya çevir
period_map = {'AM':0, 'MD':1, 'PM':2}
ml_df['period_num'] = ml_df['period'].map(period_map)

# Ayı sayıya çevir
month_map = {
    'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
    'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12
}

ml_df['month_num'] = ml_df['month'].map(month_map)

ml_df[['month','month_num','period','period_num']].head()


,month,month_num,period,period_num
3,May,5,AM,0.0
4,May,5,AM,0.0
5,May,5,AM,0.0
6,May,5,AM,0.0
8,May,5,AM,0.0


In [17]:
# Latitude / Longitude çıkarma
import re

def extract_lat_lon(point):
    match = re.search(r'POINT \(([-\d\.]+) ([-\d\.]+)\)', point)
    if match:
        lon = float(match.group(1))
        lat = float(match.group(2))
        return lat, lon
    return None, None

ml_df['lat'], ml_df['lon'] = zip(*ml_df['the_geom'].apply(extract_lat_lon))

ml_df[['the_geom','lat','lon']].head()


KeyError: 'the_geom'

In [18]:
# the_geom'u tekrar ekleyelim
ml_df = clean_df[[
    'Borough',
    'month',
    'year',
    'period',
    'pedestrian_count',
    'crowd_level',
    'the_geom'
]].copy()

ml_df.head()


,Borough,month,year,period,pedestrian_count,crowd_level,the_geom
3,Bronx,May,2007,AM,764.0,High,POINT (-73.87892467324478 40.8812869959873)
4,Bronx,May,2007,AM,650.0,High,POINT (-73.88956389732787 40.844636776717664)
5,Bronx,May,2007,AM,489.0,Medium,POINT (-73.92087824799184 40.83015855511035)
6,Bronx,May,2007,AM,703.0,High,POINT (-73.91635737290386 40.81688745834867)
8,Brooklyn,May,2007,AM,856.0,High,POINT (-74.01119296689038 40.64426682358741)


In [19]:
import re

def extract_lat_lon(point):
    match = re.search(r'POINT \(([-\d\.]+) ([-\d\.]+)\)', point)
    if match:
        lon = float(match.group(1))
        lat = float(match.group(2))
        return lat, lon
    return None, None

ml_df['lat'], ml_df['lon'] = zip(*ml_df['the_geom'].apply(extract_lat_lon))

ml_df[['the_geom','lat','lon']].head()


,the_geom,lat,lon
3,POINT (-73.87892467324478 40.8812869959873),40.881287,-73.878925
4,POINT (-73.88956389732787 40.844636776717664),40.844637,-73.889564
5,POINT (-73.92087824799184 40.83015855511035),40.830159,-73.920878
6,POINT (-73.91635737290386 40.81688745834867),40.816887,-73.916357
8,POINT (-74.01119296689038 40.64426682358741),40.644267,-74.011193


In [21]:
# Final feature set
final_X = ml_df[[
    'year',
    'month_num',
    'period_num',
    'lat',
    'lon'
]]

y = ml_df['crowd_level']

final_X.head()

KeyError: "['month_num', 'period_num'] not in index"

In [22]:
# Period ve month tekrar sayıya çevir
period_map = {'AM':0, 'MD':1, 'PM':2}
ml_df['period_num'] = ml_df['period'].map(period_map)

month_map = {
    'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
    'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12
}

ml_df['month_num'] = ml_df['month'].map(month_map)

# Kontrol
ml_df[['month','month_num','period','period_num']].head()


,month,month_num,period,period_num
3,May,5,AM,0.0
4,May,5,AM,0.0
5,May,5,AM,0.0
6,May,5,AM,0.0
8,May,5,AM,0.0


In [32]:
# Final feature set
final_X = ml_df[[
    'year',
    'month_num',
    'period_num',
    'lat',
    'lon'
]]

y = ml_df['crowd_level']


In [33]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Split
X_train, X_test, y_train, y_test = train_test_split(
    final_X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Model
final_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    random_state=42
)

# Train
final_model.fit(X_train, y_train)

# Predict
y_pred = final_model.predict(X_test)

# Evaluate
print("Final Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Final Accuracy: 0.7615894039735099
              precision    recall  f1-score   support

        High       0.74      0.77      0.76       151
         Low       0.84      0.89      0.86       151
      Medium       0.69      0.62      0.66       151

    accuracy                           0.76       453
   macro avg       0.76      0.76      0.76       453
weighted avg       0.76      0.76      0.76       453



In [34]:
# Test setine tahminleri ekle
test_df = X_test.copy()

test_df['predicted_level'] = y_pred
test_df['true_level'] = y_test.values

# Lat/lon tekrar ekleyelim
test_df['lat'] = ml_df.loc[X_test.index, 'lat']
test_df['lon'] = ml_df.loc[X_test.index, 'lon']

test_df.head()


,year,month_num,period_num,lat,lon,predicted_level,true_level
7609,2025,5,0.0,40.700488,-73.901927,Medium,Medium
3177,2016,5,0.0,40.637933,-74.077403,High,Medium
7457,2024,10,1.0,40.708063,-74.004518,High,High
451,2008,5,0.0,40.814131,-73.933133,Low,Low
6844,2000,6,0.0,40.844637,-73.889564,High,Medium


In [35]:
!pip install folium

In [36]:
import folium

# NYC merkezine yakın başlangıç
m = folium.Map(location=[40.75, -73.98], zoom_start=11)

# Renk fonksiyonu
def get_color(level):
    if level == 'Low':
        return 'green'
    elif level == 'Medium':
        return 'orange'
    else:
        return 'red'

# Haritaya noktaları ekle
for _, row in test_df.iterrows():
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=6,
        color=get_color(row['predicted_level']),
        fill=True,
        fill_opacity=0.7,
        popup=f"""
        Predicted: {row['predicted_level']}<br>
        Actual: {row['true_level']}<br>
        Year: {int(row['year'])}
        """
    ).add_to(m)

m


In [38]:
# Year sütununu temizle
ml_df['year'] = pd.to_numeric(ml_df['year'], errors='coerce')

# Bozuk olanları sil
ml_df = ml_df.dropna(subset=['year'])

# int yap
ml_df['year'] = ml_df['year'].astype(int)